In [49]:
import tensorflow as tf
import tensorflow_io as tfio
import pandas as pd
from PIL import Image
from pprint import pprint


def parse_img(d):
    d["img"] = tfio.experimental.image.decode_tiff(tf.io.read_file(d["image_file"]))
#     d["encoding"] = tf.strings.to_number(tf.strings.split(d["encoding"], " "), tf.int64)
    d["mask"] = rle_decode_tf(d["encoding"], (d["width_pixels"], d["height_pixels"]))
    return d


@tf.function
def rle_decode_tf(mask_rle, shape):
    size = tf.math.reduce_prod(shape)
    # Split string
    s = tf.strings.split(mask_rle)
    s = tf.strings.to_number(s, tf.int64)
    # Get starts and lengths
    starts = s[::2] - 1
    lens = s[1::2]
    # Make ones to be scattered
    total_ones = tf.reduce_sum(lens)
    ones = tf.ones([total_ones], tf.uint8)
    # Make scattering indices
    r = tf.range(total_ones)
    lens_cum = tf.math.cumsum(lens)
    s = tf.searchsorted(lens_cum, r, "right")
    idx = r + tf.gather(starts - tf.pad(lens_cum[:-1], [(1, 0)]), s)
    # Scatter ones into flattened mask
    mask_flat = tf.scatter_nd(tf.expand_dims(idx, 1), ones, [size])
    # Reshape into mask
    return tf.reshape(mask_flat, shape)

    
metadata = pd.read_csv("../data/HuBMAP-20-dataset_information.csv")
metadata["id"] = metadata["image_file"].str.rstrip(".tiff")
targets = pd.read_csv("../data/train.csv")
targets = targets.merge(metadata, on="id")
for k in ["image_file", "anatomical_structures_segmention_file", "glomerulus_segmentation_file"]:
    targets[k] = "../data/train/" + targets[k]
data = tf.data.Dataset.from_tensor_slices(dict(targets))
data = data.map(parse_img)
for i in data.take(1).as_numpy_iterator():
    print(i["mask"].mean())

0.017945099862100845


In [41]:
targets

,id,encoding,image_file,width_pixels,height_pixels,anatomical_structures_segmention_file,glomerulus_segmentation_file,patient_number,race,ethnicity,sex,age,weight_kilograms,height_centimeters,bmi_kg/m^2,laterality,percent_cortex,percent_medulla
0,aaa6a05cc,30989109 59 31007591 64 31026074 68 31044556 7...,../data/train/aaa6a05cc.tiff,13013,18484,../data/train/aaa6a05cc-anatomical-structure.json,../data/train/aaa6a05cc.json,65631,White,Not Hispanic or Latino,Female,73,NaN,NaN,NaN,Left,75,25
1,cb2d976f4,78144363 5 78179297 15 78214231 25 78249165 35...,../data/train/cb2d976f4.tiff,49548,34940,../data/train/cb2d976f4-anatomical-structure.json,../data/train/cb2d976f4.json,67548,Black or African American,Not Hispanic or Latino,Male,58,79.9,190.5,22.0,Right,75,25
2,0486052bb,101676003 6 101701785 8 101727568 9 101753351 ...,../data/train/0486052bb.tiff,34937,25784,../data/train/0486052bb-anatomical-structure.json,../data/train/0486052bb.json,67177,White,Not Hispanic or Latino,Male,31,106.1,180.3,32.6,Right,80,20
3,e79de561c,7464094 14 7480273 41 7496453 67 7512632 82 75...,../data/train/e79de561c.tiff,27020,16180,../data/train/e79de561c-anatomical-structure.json,../data/train/e79de561c.json,67026,Black or African American,Not Hispanic or Latino,Male,53,73.0,166.0,26.5,Left,55,45
4,54f2eec69,124601765 36 124632133 109 124662536 147 12469...,../data/train/54f2eec69.tiff,22240,30440,../data/train/54f2eec69-anatomical-structure.json,../data/train/54f2eec69.json,67548,Black or African American,Not Hispanic or Latino,Male,58,79.9,190.5,22.0,Right,75,25
5,1e2425f28,49453112 7 49479881 22 49506657 31 49533433 40...,../data/train/1e2425f28.tiff,32220,26780,../data/train/1e2425f28-anatomical-structure.json,../data/train/1e2425f28.json,63921,White,Not Hispanic or Latino,Male,48,131.5,193.0,35.3,Right,65,35


In [30]:
for i in data.as_numpy_iterator():
    print(i.keys())
    break

dict_keys(['id', 'encoding', 'image_file', 'width_pixels', 'height_pixels', 'anatomical_structures_segmention_file', 'glomerulus_segmentation_file', 'patient_number', 'race', 'ethnicity', 'sex', 'age', 'weight_kilograms', 'height_centimeters', 'bmi_kg/m^2', 'laterality', 'percent_cortex', 'percent_medulla', 'img'])


<function pandas.io.json._normalize._json_normalize(data: Union[Dict, List[Dict]], record_path: Union[str, List, NoneType] = None, meta: Union[str, List[Union[str, List[str]]], NoneType] = None, meta_prefix: Union[str, NoneType] = None, record_prefix: Union[str, NoneType] = None, errors: str = 'raise', sep: str = '.', max_level: Union[int, NoneType] = None) -> 'DataFrame'>